In [1]:
import csv
import pandas as pd
import json

df_u_data = pd.read_csv("./dataset/lastfm/user_artists.dat", delimiter='\t', engine='python')
df_artists = pd.read_csv("./dataset/lastfm/artists.dat", delimiter='\t', engine='python', quoting=csv.QUOTE_NONE)
df_u_timestamp = pd.read_csv("./dataset/lastfm/user_taggedartists-timestamps.dat", delimiter='\t', engine='python')
df_tags = pd.read_csv("./dataset/lastfm/tags.dat", delimiter='\t', engine='python', encoding='ISO-8859-1')

In [10]:
xxxxx = df_u_timestamp[['userID', 'artistID']]
xxxxx.duplicated()
xxxxx = xxxxx.drop_duplicates(subset=['userID', 'artistID'], keep='first')
tmp_dataset = xxxxx.values.tolist()

In [15]:
result = {}
for user, item in tmp_dataset:
    if user in result.keys():
        result[user].append(item)
    else:
        result[user] = [item]
len(result)

1892

In [16]:
len_data = []
for user in result.keys():
    len_data.append(len(result[user]))
len_data.sort()
len_data.index(21)
# len(len_data)

1297

In [17]:
data = df_artists[['id','name']].values.tolist()
with open('lf_name2id.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)

In [38]:
user_id = 2
u_data_list = df_u_data[df_u_data.userID == user_id].values.tolist()[:20]


for _, artist_id, weight in u_data_list:
    tag_list = df_u_timestamp[(df_u_timestamp.artistID == artist_id) & (df_u_timestamp.userID == user_id)].tagID.values.tolist()
    tag_list = df_tags[df_tags.tagID.isin(tag_list)].tagValue.values.tolist()
    if len(tag_list) != 0:
        tag_from_user = str.join('|', tag_list)
        print(f"Artist ID: {df_artists[df_artists.id == artist_id].name.values[0]}, Artist Weight: {weight}, Tag Name: {tag_from_user}")
    else:
        print(f"Artist ID: {df_artists[df_artists.id == artist_id].name.values[0]}, Artist Weight: {weight}")

Artist ID: Duran Duran, Artist Weight: 13883
Artist ID: Morcheeba, Artist Weight: 11690, Tag Name: chillout|downtempo|electronic|trip-hop|female vovalists
Artist ID: Air, Artist Weight: 11351
Artist ID: Hooverphonic, Artist Weight: 10300
Artist ID: Kylie Minogue, Artist Weight: 8983
Artist ID: Daft Punk, Artist Weight: 6152
Artist ID: Thievery Corporation, Artist Weight: 5955
Artist ID: Goldfrapp, Artist Weight: 4616
Artist ID: New Order, Artist Weight: 4337
Artist ID: Matt Bianco, Artist Weight: 4147
Artist ID: Talk Talk, Artist Weight: 3923
Artist ID: Prefab Sprout, Artist Weight: 3782
Artist ID: Enigma, Artist Weight: 3735, Tag Name: chillout|ambient|new age|gregorian chant
Artist ID: Röyksopp, Artist Weight: 3644
Artist ID: Coldplay, Artist Weight: 3579
Artist ID: Faithless, Artist Weight: 3312
Artist ID: Madonna, Artist Weight: 3301
Artist ID: Icehouse, Artist Weight: 2927
Artist ID: Sade, Artist Weight: 2720
Artist ID: Moby, Artist Weight: 2686


In [ ]:
# generate training data
import random
user_count = 0

with open('training_data/train_lf_with_userid.json', 'w', encoding='utf-8') as fj:
    llm = []
    irs = []
    
    for user_id in range(1, 2101):
        print(user_id)
        df_user = df_u_timestamp[df_u_timestamp.userID == user_id]
        u_artists_list = df_user.sort_values(by='timestamp').artistID.values.tolist()
        
        if len(u_artists_list) <= 20 or len(set(df_user.timestamp.values.tolist())) <= 5:
            # print(user_id)
            continue
        # print(len(df_user))
        uid_his_tra = []
        for _ in u_artists_list:
            if _ not in uid_his_tra:
                uid_his_tra.append(_)
        
        if len(uid_his_tra) <= 10:
            continue
        
        prompt_his = ""
        uid_his = []
        for artist_id in uid_his_tra[-20:]:
            if len(df_u_data[(df_u_data.userID == user_id) & (df_u_data.artistID == artist_id)].values) == 0:
                continue
            # print(f"{df_artists[df_artists.id == artist_id].name.values[0]}, Listening count: {df_u_data[(df_u_data.userID == user_id) & (df_u_data.artistID == artist_id)].weight.values[0]}")
            prompt_his += f"{df_artists[df_artists.id == artist_id].name.values[0]}, Listening count: {df_u_data[(df_u_data.userID == user_id) & (df_u_data.artistID == artist_id)].weight.values[0]}\n"
            uid_his.append(artist_id)
        
        if len(uid_his) <= 8:
            continue
        
        while True:
            artist_tar = random.randint(1,18745)
            if artist_tar not in uid_his_tra and len(df_artists[df_artists.id == artist_tar].values.tolist()) != 0:
                break
    
        artist_tar_name = df_artists[df_artists.id == artist_tar].name.values[0]
        # print(f"Target artist: {artist_tar_name}")
        
        llm.append([artist_tar_name, prompt_his])
        irs.append([artist_tar, uid_his_tra[-20:], user_id])
        
        user_count += 1
        if user_count == 100:
            print('finish!')
            break
        # break
    # print(llm)
    print(user_count)
    json.dump({'llm':llm,'irs':irs}, fj)

In [9]:
#main code
import random
user_count = 0

with open('training_data/train_lf_with_userid.json', 'w', encoding='utf-8') as fj:
    llm = []
    irs = []
    
    for user_id in range(1, 2101):
        print(user_id)
        df_user = df_u_timestamp[df_u_timestamp.userID == user_id]
        u_artists_list = df_user.sort_values(by='timestamp').artistID.values.tolist()
        
        if len(u_artists_list) <= 20 or len(set(df_user.timestamp.values.tolist())) <= 5:
            # print(user_id)
            continue
        # print(len(df_user))
        uid_his_tra = []
        for _ in u_artists_list:
            if _ not in uid_his_tra:
                uid_his_tra.append(_)
        
        if len(uid_his_tra) <= 10:
            continue
        
        prompt_his = ""
        uid_his = []
        for artist_id in uid_his_tra[-20:]:
            if len(df_u_data[(df_u_data.userID == user_id) & (df_u_data.artistID == artist_id)].values) == 0:
                continue
            # print(f"{df_artists[df_artists.id == artist_id].name.values[0]}, Listening count: {df_u_data[(df_u_data.userID == user_id) & (df_u_data.artistID == artist_id)].weight.values[0]}")
            prompt_his += f"{df_artists[df_artists.id == artist_id].name.values[0]}, Listening count: {df_u_data[(df_u_data.userID == user_id) & (df_u_data.artistID == artist_id)].weight.values[0]}\n"
            uid_his.append(artist_id)
        
        if len(uid_his) <= 8:
            continue
        
        while True:
            artist_tar = random.randint(1,18745)
            if artist_tar not in uid_his_tra and len(df_artists[df_artists.id == artist_tar].values.tolist()) != 0:
                break
    
        artist_tar_name = df_artists[df_artists.id == artist_tar].name.values[0]
        # print(f"Target artist: {artist_tar_name}")
        
        llm.append([artist_tar_name, prompt_his])
        irs.append([artist_tar, uid_his_tra[-20:], user_id])
        
        user_count += 1
        if user_count == 100:
            print('finish!')
            break
        # break
    # print(llm)
    print(user_count)
    json.dump({'llm':llm,'irs':irs}, fj)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
for _, artist_id, weight in u_data_list:
    tag_list = u_tags[(u_tags.artistID == artist_id) & (u_tags.userID == user_id)].tagID.values.tolist()
    tag_list = tags[tags.tagID.isin(tag_list)].tagValue.values.tolist()
    if len(tag_list) != 0:
        tag_from_user = str.join('|', tag_list)
        print(f"Artist ID: {u_artists[u_artists.id == artist_id].name.values[0]}, Artist Weight: {weight}, Tag Name: {tag_from_user}")
    else:
        print(f"Artist ID: {u_artists[u_artists.id == artist_id].name.values[0]}, Artist Weight: {weight}")

In [33]:
tag_list = u_tags[(u_tags.artistID == 52) & (u_tags.userID == 2)].tagID.values.tolist()
tag_from_user = str.join('|', tags[tags.tagID.isin(tag_list)].tagValue.values.tolist())

'chillout|downtempo|electronic|trip-hop|female vovalists'

In [7]:
with open('training_data/train_irs_lf.json', 'w', encoding='utf-8') as f:
    for user_id in range(2, 2101):
        df_user = df_u_timestamp[df_u_timestamp.userID == user_id]
        u_artists_list = df_user.sort_values(by='timestamp').values.tolist()
        # print(u_artists_list)
        for _, artist_id, tag_id, timestamp in u_artists_list:
            f.write(f"{user_id} {artist_id}\n")

In [17]:
# reform irs to ori & fix

import json
# with open('training_data/output_gpt_lf_IRS_pro.json', 'r', encoding='utf-8') as f:
with open('training_data/output_gpt_lf_POP_pro.json', 'r', encoding='utf-8') as f:
    content = json.load(f)

output = []
output_fix = []
for target, his_tra, influ_path in content:
    _path = []
    target_flag = False
    if target in influ_path:
        influ_path = influ_path[:influ_path.index(target)+1]
        target_flag = True
    
    for item in influ_path:
        print(item)
        _ = df_artists[df_artists.id == item].name.values[0]
        # print(_)
        _path.append(_)
        
    output.append([_path.copy(),[]])
    if not target_flag:
        _path.append(df_artists[df_artists.id == target].name.values[0])
    output_fix.append([_path.copy(),[]])
    
    
with open(f"training_data/lastfm/output_gpt_lf_POP_pro.json", "w", encoding='utf-8') as f:
    json.dump(output, f)    
# with open(f"training_data/lastfm/output_gpt_lf_IRSori_pro.json", "w", encoding='utf-8') as f:
#     json.dump(output, f)
# with open(f"training_data/lastfm/output_gpt_lf_IRSfix_pro.json", "w", encoding='utf-8') as f:
#     json.dump(output_fix, f)

4394
2626
1643
1772
1620
360
306
310
301
302
320
318
325
326
333
377
378
405
412
430
317
314
302
306
310
301
299
300
344
461
466
292
289
288
257
285
291
294
295
1464
220
227
234
230
233
238
229
228
89
289
292
295
288
257
285
291
294
544
808
815
301
302
306
310
299
300
344
461
466
292
289
288
55
67
283
265
227
229
228
217
288
257
285
291
310
301
302
306
298
300
344
461
466
524
543
523
525
538
540
779
957
959
1249
1412
1415
2834
1813
17652
1815
1816
1817
1873
982
961
1112
1098
964
206
203
190
1274
1118
1055
504
518
517
511
65
67
89
300
301
302
306
310
299
295
292
257
288
207
210
220
227
234
230
233
238
229
228
89
289
292
295
288
257
285
291
294
544
182
180
153
99
85
77
51
993
1071
1075
1155
1122
1110
1125
3481
1123
1239
2786
3203
862
863
858
163
159
154
190
173
172
225
227
234
230
233
238
229
228
89
289
292
295
751
852
612
1017
8241
601
599
735
198
209
220
227
234
230
233
238
229
228
89
289
966
959
1249
1412
1415
2834
1813
17652
1815
1816
1817
1873
982
961
1112
1098
964
206
203
190
67
89

In [16]:
#POP
import json
from collections import Counter
from random import choice

with open("training_data/train_lf.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

artist_lst = df_artists.id.values.tolist()

total_movie_rec = []
movie_chain = {}
for user_id in range(1, 2101):
    print(user_id)
    df_user = df_u_timestamp[df_u_timestamp.userID == user_id]
    
    if len(df_user) == 0:
        continue
    
    u_artists_list = df_user.sort_values(by='timestamp').artistID.values.tolist()
    
    uid_his_tra = []
    for _ in u_artists_list:
        if _ not in uid_his_tra:
            uid_his_tra.append(_)
    
    df_user = uid_his_tra
    
    cur = None
    for nxt in df_user:
        if cur == 7834:
            print(cur)
        if cur is None:
            cur = nxt
            continue
            
        if nxt in artist_lst and cur in artist_lst:
            total_movie_rec.append(nxt)
            if cur in movie_chain.keys():
                movie_chain[cur].append(nxt)
            else:
                movie_chain[cur] = [nxt]
                
        cur = nxt

def lst2counter(lst):
    tmp = Counter(lst)
    result = sorted(tmp.items(), key=lambda x: x[1], reverse=True)
    return [_[0] for _ in result]

for key in movie_chain.keys():
    tra = movie_chain[key]
    movie_chain[key] = lst2counter(tra)

total_movie_rec = lst2counter(total_movie_rec)

def top_k(a, p):
    top_k_list = a
    for i in top_k_list:
        if i in p:
            continue
        return i
    return top_k(total_movie_rec, p)

# def random_choice(p):
#     while True:
#         tmp = choice(movies)
#         if tmp[0] not in p:
#             return tmp[0]

SR = 0
path_lst = []
for tar, his_tra in data['irs']:
    influ_path = []
    cur = his_tra[-1]
    for _ in range(20):
        if cur == 7834:
            print(cur)
        if cur not in movie_chain.keys():
            nxt = top_k(total_movie_rec, his_tra+influ_path)
        else:
            nxt = top_k(movie_chain[cur], his_tra+influ_path)
        influ_path.append(nxt)
        cur = nxt
    if tar in influ_path:
        SR += 1
    path_lst.append([tar,his_tra,influ_path.copy()])

with open(f"training_data/output_gpt_lf_POP_pro.json", "w", encoding='utf-8') as f:
    json.dump(path_lst, f)
print(SR/len(data['irs']))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
7834
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276